<a href="https://colab.research.google.com/github/Talha1818/Machine-Learning-Python-Notebooks/blob/master/Train_Model_Single_Page_BERT_(Final_BERT_%2B_Post_ALGO_%2B_Single_Page_BERT).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Annotation Folder/Final_Text_Dataset_10_March_2023.csv")
df.head()

,page#,package_name,text,token,clean_text
0,1.0,612007_NJ_CAMDEN,zp wo o\ngD ong 8 8 n xmwor\n3 5 ee S S ee nN\...,15.0,ong xmwor nnupoo sko uuuuuuu aneaf uuuuu tssar...
1,2.0,612007_NJ_CAMDEN,|\n[gem a CASHIER'S CHECK 6780001482\n| Remite...,15.0,gem cashiers check remiter salvador carrasco o...
2,3.0,612007_NJ_CAMDEN,"City Abstract, LLC ALTA Buyer's Final Settleme...",11.0,city abstract llc alta buyers final settlement...
3,4.0,612007_NJ_CAMDEN,Po Description\nTile Platform Fee (TMP) to Ciy...,11.0,description tile platform fee tmp ciyabstrack ...
4,5.0,612007_NJ_CAMDEN,"City Abstract, LLC ALTA Seller's Settlement St...",13.0,city abstract llc alta sellers settlement stat...


In [ ]:
df = df[['clean_text', 'token']]
df.head()

,clean_text,token
0,ong xmwor nnupoo sko uuuuuuu aneaf uuuuu tssar...,15.0
1,gem cashiers check remiter salvador carrasco o...,15.0
2,city abstract llc alta buyers final settlement...,11.0
3,description tile platform fee tmp ciyabstrack ...,11.0
4,city abstract llc alta sellers settlement stat...,13.0


In [ ]:
df = df[df['token'].isin([27, 29, 36, 38, 39, 44, 45, 614, 63, 71, 74,
                76, 79, 80, 82, 84, 100, 103, 109, 110, 113, 117,
                138, 197, 203, 75, 170, 102, 130, 160, 168, 183, 224, 228, 257])]

In [ ]:
df['token'].value_counts()

44.0     683
36.0     432
45.0     347
39.0     324
168.0    253
29.0     233
76.0     216
113.0    203
27.0     134
100.0    128
71.0     123
63.0     115
75.0      96
257.0     91
170.0     91
84.0      88
79.0      87
203.0     80
117.0     78
130.0     63
74.0      62
224.0     60
38.0      55
110.0     46
138.0     41
614.0     38
103.0     31
102.0     26
160.0     22
80.0      22
109.0     22
82.0      21
183.0     17
197.0     16
228.0     15
Name: token, dtype: int64

In [ ]:
df['token'] = df['token'].astype(int)

In [ ]:
df = df[df['clean_text'].notna()]
df = df[df['token'].notna()]

In [ ]:
df['clean_text'] = "<START> " + df['clean_text'] + " <END> <DOCUMENT>"

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Create label encoder object
label_encoder = LabelEncoder()

# Fit label encoder and transform labels
df['token'] = label_encoder.fit_transform(df['token'])
df['token'].value_counts()

5     683
2     432
6     347
4     324
26    253
1     233
11    216
21    203
0     134
16    128
8     123
7     115
10     96
33     91
27     91
15     88
12     87
30     80
22     78
23     63
9      62
31     59
3      55
20     46
24     41
34     38
18     31
17     26
25     22
13     22
19     22
14     21
28     17
29     16
32     15
Name: token, dtype: int64

In [ ]:
# Get mapping of label encoding values to original classes
label_encoding_mapping = dict(zip(label_encoder.transform(label_encoder.classes_), label_encoder.classes_))
print(label_encoding_mapping)

{0: 27, 1: 29, 2: 36, 3: 38, 4: 39, 5: 44, 6: 45, 7: 63, 8: 71, 9: 74, 10: 75, 11: 76, 12: 79, 13: 80, 14: 82, 15: 84, 16: 100, 17: 102, 18: 103, 19: 109, 20: 110, 21: 113, 22: 117, 23: 130, 24: 138, 25: 160, 26: 168, 27: 170, 28: 183, 29: 197, 30: 203, 31: 224, 32: 228, 33: 257, 34: 614}


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 55.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 108.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 KB 21.5 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
from tqdm.auto import tqdm

In [ ]:
processed_data = []
for i in tqdm(range(len(df))):
  text = df['clean_text'].iloc[i]
  label = df['token'].iloc[i]

  encodings = tokenizer(text, padding="max_length", truncation=True, max_length=512,
                      add_special_tokens=True, # Add [CLS] and [SEP] tokens
                      )
  encodings['label'] = label
  encodings['text'] = text
  processed_data.append(encodings)

  0%|          | 0/4358 [00:00<?, ?it/s]

In [ ]:
from sklearn.model_selection import train_test_split

new_df = pd.DataFrame(processed_data)

train_df, valid_df = train_test_split(
    new_df,
    test_size=0.2,
    random_state=2023
)

In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 KB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 37.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 13.2 MB/s eta 0:00:00


In [ ]:
import pyarrow as pa
from datasets import Dataset

train_hg = Dataset(pa.Table.from_pandas(train_df))
valid_hg = Dataset(pa.Table.from_pandas(valid_df))

In [ ]:
len(df['token'].unique())

35

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=len(df['token'].unique())
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = accuracy_score(labels, predictions)
    precision, recall, _, _ = precision_recall_fscore_support(labels, predictions, average='weighted')
    return {'accuracy': accuracy, 'precision': precision, 'recall': recall}

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="/content/drive/MyDrive/Annotation Folder/RESULTS_SINGLE_PAGE",
                                  evaluation_strategy="epoch",
                                  num_train_epochs=3,  # set the number of epochs here
                                  # resume_from_checkpoint="/content/drive/MyDrive/Annotation Folder/RESULTS_SINGLE_PAGE/checkpoint-7000",
                                  save_strategy="steps",  # save checkpoints every epoch
                                  save_steps=1000,  # save checkpoints every 10 epochs
                                  )

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_hg,
    eval_dataset=valid_hg,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall
1,No log,0.299157,0.949541,0.927609,0.949541
2,1.060100,0.166285,0.966743,0.957854,0.966743
3,0.177900,0.145935,0.971330,0.973737,0.971330


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TrainOutput(global_step=1308, training_loss=0.4987284867406256, metrics={'train_runtime': 1098.5808, 'train_samples_per_second': 9.52, 'train_steps_per_second': 1.191, 'total_flos': 2752430703224832.0, 'train_loss': 0.4987284867406256, 'epoch': 3.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.14593517780303955,
 'eval_accuracy': 0.9713302752293578,
 'eval_precision': 0.9737374799990967,
 'eval_recall': 0.9713302752293578,
 'eval_runtime': 29.9294,
 'eval_samples_per_second': 29.135,
 'eval_steps_per_second': 3.642,
 'epoch': 3.0}

In [ ]:
model.save_pretrained('/content/drive/MyDrive/Annotation Folder/BertModelFiles/BERT_MODEL_SINGLE_PAGE_DOC/')

In [ ]:
from transformers import AutoTokenizer

new_tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

# Save tokenizer to disk
new_tokenizer.save_pretrained('/content/drive/MyDrive/Annotation Folder/BertModelFiles/TOKENIZER_SINGLE_PAGE')

('/content/drive/MyDrive/Annotation Folder/BertModelFiles/TOKENIZER_SINGLE_PAGE/tokenizer_config.json',
 '/content/drive/MyDrive/Annotation Folder/BertModelFiles/TOKENIZER_SINGLE_PAGE/special_tokens_map.json',
 '/content/drive/MyDrive/Annotation Folder/BertModelFiles/TOKENIZER_SINGLE_PAGE/vocab.txt',
 '/content/drive/MyDrive/Annotation Folder/BertModelFiles/TOKENIZER_SINGLE_PAGE/added_tokens.json',
 '/content/drive/MyDrive/Annotation Folder/BertModelFiles/TOKENIZER_SINGLE_PAGE/tokenizer.json')

# **Model Inference**

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

new_model = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/Annotation Folder/BertModelFiles/BERT_MODEL_SINGLE_PAGE_DOC/')
new_tokenizer = AutoTokenizer.from_pretrained('/content/drive/MyDrive/Annotation Folder/BertModelFiles/TOKENIZER_SINGLE_PAGE')

In [ ]:
import torch
import torch.nn.functional as F
import numpy as np

actual_label = {0: 27, 1: 29, 2: 36, 3: 38, 4: 39, 5: 44, 6: 45, 7: 63, 8: 71, 9: 74, 10: 75, 11: 76, 12: 79, 13: 80, 14: 82, 15: 84, 16: 100, 17: 102, 18: 103, 19: 109, 20: 110, 21: 113, 22: 117, 23: 130, 24: 138, 25: 160, 26: 168, 27: 170, 28: 183, 29: 197, 30: 203, 31: 224, 32: 228, 33: 257, 34: 614}
actual_label = {k: str(v) for k, v in actual_label.items()}

import psutil

def get_prediction_SINGLE_PAGE(text):
    encoding = new_tokenizer(text, return_tensors="pt", padding="max_length", truncation=True, max_length=512,
                      add_special_tokens=True, # Add [CLS] and [SEP] tokens
                      )
    encoding = {k: v for k,v in encoding.items()}

    # Get initial CPU and RAM usage
    process = psutil.Process()
    initial_cpu = process.cpu_times()

    initial_ram = process.memory_info().rss / 1e6

    outputs = new_model(**encoding)

    logits = outputs.logits

    probs = F.softmax(logits, dim=1).squeeze().cpu().detach().numpy()

    label = np.argmax(probs, axis=-1)

    # Get CPU and RAM usage after running the model
    cpu_usage = sum(process.cpu_times()) - sum(initial_cpu)
    ram_usage = (process.memory_info().rss / 1e6) - initial_ram
    
    return {
        'label': int(actual_label[label]),
        'probability': max(probs),
        'cpu_usage (seconds)': cpu_usage,
        'ram_usage (MB)': ram_usage
    }


In [ ]:
import re
import pandas as pd
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
import re


# Define regex patterns
punct_pattern = r'[^\w\s]'
num_pattern = r'\d+'
special_pattern = r'[^A-Za-z0-9\s]'

# Define cleaning function
def clean_text(text):
    text = re.sub(r'\\n|\n', ' ', text)
    # remove links
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r'\x00|\\x00', '', text)
    # Convert to lowercase
    text = text.lower()
    # Remove punctuation
    text = re.sub(punct_pattern, '', text)
    # Remove integers
    text = re.sub(num_pattern, '', text)
    # Remove special characters
    text = re.sub(special_pattern, '', text)
    # Remove extra whitespace
    text = re.sub('\s+', ' ', text).strip()
    # Remove nextline
    text = re.sub(r'\n', ' ', text)
    # Remove words with length 1
    text = ' '.join(word for word in text.split() if len(word) > 2)
    # Remove stop words
    stop_words = set(stopwords.words('english'))
    text = ' '.join(word for word in text.split() if word not in stop_words)
    return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# text = """HELLO """

# prepare_text = "<START> " + text + " <END> <DOCUMENT>"
# input_text = clean_text(prepare_text)
# input_text

In [ ]:
# get_prediction_SINGLE_PAGE(input_text)

# **PREDICTION**

In [ ]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/Annotation Folder/Few_Packages/Sample8Packages.csv")
df = df[~df['package_name'].isin(['Raheem_SAMPLE_CHECKING', 'Sample_9_NJ'])]
df.head()

,page#,package_name,text,token,our_prediction
105,1.0,218248_NJ_CAMDEN,File No.: CAHD-612066\nAFFIDAVIT OF TITLE\nPUR...,3.0,NaN
106,2.0,218248_NJ_CAMDEN,| File No.: CAHD-612066\n| 7. Exceptions and A...,3.0,NaN
107,3.0,218248_NJ_CAMDEN,"City Abstract, LLC ALTA Buyer's Settlement Sta...",11.0,NaN
108,4.0,218248_NJ_CAMDEN,Description\na Credit\nTitle - Lender's NJRB 5...,11.0,NaN
109,5.0,218248_NJ_CAMDEN,"City Abstract, LLC ALTA Seller's Settlement St...",13.0,NaN


In [ ]:
df['package_name'].value_counts()

218245_NJ_CAMDEN    100
218246_NJ_SALEM      92
218248_NJ_CAMDEN     90
117665_NJ_CAMDEN     85
Name: package_name, dtype: int64

In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer

first_model = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/Annotation Folder/BertModelFiles/BERT_MODEL_FULL_SAMPLE_GREATER_100')
first_tokenizer = AutoTokenizer.from_pretrained('/content/drive/MyDrive/Annotation Folder/BertModelFiles/tokenizer')


second_single_page_model = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/Annotation Folder/BertModelFiles/BERT_MODEL_SINGLE_PAGE_DOC/')
second_single_page_model_tokenizer = AutoTokenizer.from_pretrained('/content/drive/MyDrive/Annotation Folder/BertModelFiles/TOKENIZER_SINGLE_PAGE')

# **Bert Model on 100 Frequency Documents**

In [ ]:
import torch
import torch.nn.functional as F
import numpy as np

actual_label__ = {0: 0, 1: 1, 2: 2, 3: 3, 4: 11,
                5: 12, 6: 13, 7: 14, 8: 15, 9: 20, 
                10: 21, 11: 22, 12: 23, 13: 24, 14: 25, 
                15: 27, 16: 28, 17: 29, 18: 30, 19: 32, 20: 33,
                21: 34, 22: 35, 23: 36, 24: 37, 25: 39, 26: 40, 27: 41, 28: 42, 29: 43, 
                30: 44, 31: 45, 32: 46, 33: 49, 34: 50, 35: 51, 36: 52, 37: 53, 38: 54, 39: 55, 
                40: 62, 41: 63, 42: 64, 43: 65, 44: 66, 45: 67, 46: 68, 47: 69, 48: 70, 49: 71, 50: 72,
                51: 73, 52: 75, 53: 76, 54: 81, 55: 90, 56: 93, 57: 94, 58: 95, 59: 96, 60: 98, 61: 99, 
                62: 100, 63: 111, 64: 112, 65: 113, 66: 118, 67: 123, 68: 133, 69: 153, 70: 156, 71: 165, 
                72: 166, 73: 168, 74: 170, 75: 185, 76: 214, 77: 222, 78: 223, 79: 225, 80: 239, 81: 242, 
                82: 288, 83: 289, 84: 322, 85: 325, 86: 333, 87: 348, 88: 390, 89: 448, 90: 451, 91: 521, 92: 927}

actual_label__ = {k: str(v) for k, v in actual_label__.items()}

In [ ]:
import psutil


PACKAGE_PREDICTION = []
def get_prediction(text):
    encoding = first_tokenizer(text, return_tensors="pt", padding="max_length", truncation=True, max_length=512,
                      add_special_tokens=True, # Add [CLS] and [SEP] tokens
                      )
    encoding = {k: v for k,v in encoding.items()}

    # Get initial CPU and RAM usage
    process = psutil.Process()
    initial_cpu = process.cpu_times()

    initial_ram = process.memory_info().rss / 1e6

    outputs = first_model(**encoding)
    

    logits = outputs.logits
  
    probs = F.softmax(logits, dim=1).squeeze().cpu().detach().numpy()
    

    label = np.argmax(probs, axis=-1)

    # Get CPU and RAM usage after running the model
    cpu_usage = sum(process.cpu_times()) - sum(initial_cpu)
    ram_usage = (process.memory_info().rss / 1e6) - initial_ram
    result = {
        'label': int(actual_label__[label]),
        'probability': max(probs),
        'cpu_usage (seconds)': cpu_usage,
        'ram_usage (MB)': ram_usage
    }
    PACKAGE_PREDICTION.append(result)
    return result, logits

# **Bert Model on Single Page Documents**

In [ ]:
import torch
import torch.nn.functional as F
import numpy as np

actual_label = {0: 27, 1: 29, 2: 36, 3: 38, 4: 39, 5: 44, 6: 45, 7: 63, 8: 71, 9: 74, 10: 75, 11: 76, 12: 79, 13: 80, 14: 82, 15: 84, 16: 100, 17: 102, 18: 103, 19: 109, 20: 110, 21: 113, 22: 117, 23: 130, 24: 138, 25: 160, 26: 168, 27: 170, 28: 183, 29: 197, 30: 203, 31: 224, 32: 228, 33: 257, 34: 614}
actual_label = {k: str(v) for k, v in actual_label.items()}

import psutil

def get_prediction_SINGLE_PAGE(text):
    encoding = second_single_page_model_tokenizer(text, return_tensors="pt", padding="max_length", truncation=True, max_length=512,
                      add_special_tokens=True, # Add [CLS] and [SEP] tokens
                      )
    encoding = {k: v for k,v in encoding.items()}

    # Get initial CPU and RAM usage
    process = psutil.Process()
    initial_cpu = process.cpu_times()

    initial_ram = process.memory_info().rss / 1e6

    outputs = second_single_page_model(**encoding)

    logits = outputs.logits

    probs = F.softmax(logits, dim=1).squeeze().cpu().detach().numpy()

    label = np.argmax(probs, axis=-1)

    # Get CPU and RAM usage after running the model
    cpu_usage = sum(process.cpu_times()) - sum(initial_cpu)
    ram_usage = (process.memory_info().rss / 1e6) - initial_ram
    
    return {
        'label': int(actual_label[label]),
        'probability': max(probs),
        'cpu_usage (seconds)': cpu_usage,
        'ram_usage (MB)': ram_usage
    }


### **Text Cleaning**

In [ ]:
import re
import pandas as pd
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
import re


# Define regex patterns
punct_pattern = r'[^\w\s]'
num_pattern = r'\d+'
special_pattern = r'[^A-Za-z0-9\s]'

# Define cleaning function
def clean_text(text):
    text = re.sub(r'\\n|\n', ' ', text)
    # remove links
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r'\x00|\\x00', '', text)
    # Convert to lowercase
    text = text.lower()
    # Remove punctuation
    text = re.sub(punct_pattern, '', text)
    # Remove integers
    text = re.sub(num_pattern, '', text)
    # Remove special characters
    text = re.sub(special_pattern, '', text)
    # Remove extra whitespace
    text = re.sub('\s+', ' ', text).strip()
    # Remove nextline
    text = re.sub(r'\n', ' ', text)
    # Remove words with length 1
    text = ' '.join(word for word in text.split() if len(word) > 2)
    # Remove stop words
    stop_words = set(stopwords.words('english'))
    text = ' '.join(word for word in text.split() if word not in stop_words)
    return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
df

,page#,package_name,text,token,our_prediction
105,1.0,218248_NJ_CAMDEN,File No.: CAHD-612066\nAFFIDAVIT OF TITLE\nPUR...,3.0,NaN
106,2.0,218248_NJ_CAMDEN,| File No.: CAHD-612066\n| 7. Exceptions and A...,3.0,NaN
107,3.0,218248_NJ_CAMDEN,"City Abstract, LLC ALTA Buyer's Settlement Sta...",11.0,NaN
108,4.0,218248_NJ_CAMDEN,Description\na Credit\nTitle - Lender's NJRB 5...,11.0,NaN
109,5.0,218248_NJ_CAMDEN,"City Abstract, LLC ALTA Seller's Settlement St...",13.0,NaN
...,...,...,...,...,...
569,81.0,117665_NJ_CAMDEN,American Land Title Association Commitment for...,1.0,NaN
570,82.0,117665_NJ_CAMDEN,American Land Title Association Commitment for...,1.0,NaN
571,83.0,117665_NJ_CAMDEN,Closing Disclosure\n\nClosing Information Tran...,12.0,NaN
572,84.0,117665_NJ_CAMDEN,. :\nClosing Cost Details\nSeller Paid\nAt Clo...,12.0,NaN


In [ ]:
from tqdm import tqdm
prediction = []
for pck, text, token, pg in tqdm(df[['package_name', 'text', 'token', 'page#']].values):
  prepare_text = "<START> " + text + " <END> <DOCUMENT>"
  input_text = clean_text(prepare_text)
  res, _ = get_prediction(input_text)
  res['page#'] = pg
  res['package_name'] = pck
  res['text'] = text
  res['token'] = token
  prediction.append(res)

100%|██████████| 367/367 [01:04<00:00,  5.73it/s]


In [ ]:
df_res = pd.DataFrame(prediction)

In [ ]:
df_res = df_res[['page#','package_name', 'text', 'token','label', 'probability', 
                 'cpu_usage (seconds)','ram_usage (MB)']]

df_res['page#'] = df_res['page#'].astype(int)

In [ ]:
df_res.head()

,page#,package_name,text,token,label,probability,cpu_usage (seconds),ram_usage (MB)
0,1,218248_NJ_CAMDEN,File No.: CAHD-612066\nAFFIDAVIT OF TITLE\nPUR...,3.0,3,0.999406,4.60,62.332928
1,2,218248_NJ_CAMDEN,| File No.: CAHD-612066\n| 7. Exceptions and A...,3.0,3,0.999380,4.17,606.781440
2,3,218248_NJ_CAMDEN,"City Abstract, LLC ALTA Buyer's Settlement Sta...",11.0,11,0.999020,4.23,62.074880
3,4,218248_NJ_CAMDEN,Description\na Credit\nTitle - Lender's NJRB 5...,11.0,11,0.999001,4.18,26.009600
4,5,218248_NJ_CAMDEN,"City Abstract, LLC ALTA Seller's Settlement St...",13.0,13,0.998211,4.14,0.000000


In [ ]:
df_token = pd.read_csv("/content/drive/MyDrive/Annotation Folder/lender_docs_token.csv")
df_token.head()

,Keys,Token,Meaning,Classification
0,CS,0.0,Cover Sheet,Lender Docs
1,MUC,1.0,MARKED UP COMMITMENT,NaN
2,DC,2.0,DEED COPY,NaN
3,AOT,3.0,AFF OF TITLE,NaN
4,IA,4.0,ID AFFIDAVIT (Buyer),NaN


In [ ]:
def get_token_meaning(df, target_name = 'target'):
  token_name = []
  for i in df[target_name]:
    try:
      token_name.append(df_token[df_token['Token']==i]['Meaning'].values[0])
    except:
      token_name.append('')
  return token_name

In [ ]:
token_name = []
for i in df_res['label']:
  try:
    token_name.append(df_token[df_token['Token']==i]['Meaning'].values[0])
  except:
    token_name.append('')

In [ ]:
df_res['Meaning'] = token_name

In [ ]:
df_res['token'] = df_res['token'].astype(int)

In [ ]:
df_res.head()

,page#,package_name,text,token,label,probability,cpu_usage (seconds),ram_usage (MB),Meaning
0,1,218248_NJ_CAMDEN,File No.: CAHD-612066\nAFFIDAVIT OF TITLE\nPUR...,3,3,0.999406,4.60,62.332928,AFF OF TITLE
1,2,218248_NJ_CAMDEN,| File No.: CAHD-612066\n| 7. Exceptions and A...,3,3,0.999380,4.17,606.781440,AFF OF TITLE
2,3,218248_NJ_CAMDEN,"City Abstract, LLC ALTA Buyer's Settlement Sta...",11,11,0.999020,4.23,62.074880,ALTA BUYER’S STATEMENT
3,4,218248_NJ_CAMDEN,Description\na Credit\nTitle - Lender's NJRB 5...,11,11,0.999001,4.18,26.009600,ALTA BUYER’S STATEMENT
4,5,218248_NJ_CAMDEN,"City Abstract, LLC ALTA Seller's Settlement St...",13,13,0.998211,4.14,0.000000,ALTA SELLER’S STATEMENT


# **Post Algo**

In [ ]:
#copying the dataframe to perform experiment
test_df = df_res.copy()


threshold = 0.97 #Setting the threshold value. Above than 96% seems to be good score

#List of documents that appear as a single page document in the package
single_list= [27, 29, 36, 38, 39, 44, 45, 614, 63, 71, 74, 
             76, 79, 80, 82, 84, 100, 103, 109, 110, 113, 117, 
             138, 197, 203, 75, 170, 102, 130, 160, 168, 183, 224, 228, 257]


In [ ]:
test_df.head()

,page#,package_name,text,token,label,probability,cpu_usage (seconds),ram_usage (MB),Meaning
0,1,218248_NJ_CAMDEN,File No.: CAHD-612066\nAFFIDAVIT OF TITLE\nPUR...,3,3,0.999406,4.60,62.332928,AFF OF TITLE
1,2,218248_NJ_CAMDEN,| File No.: CAHD-612066\n| 7. Exceptions and A...,3,3,0.999380,4.17,606.781440,AFF OF TITLE
2,3,218248_NJ_CAMDEN,"City Abstract, LLC ALTA Buyer's Settlement Sta...",11,11,0.999020,4.23,62.074880,ALTA BUYER’S STATEMENT
3,4,218248_NJ_CAMDEN,Description\na Credit\nTitle - Lender's NJRB 5...,11,11,0.999001,4.18,26.009600,ALTA BUYER’S STATEMENT
4,5,218248_NJ_CAMDEN,"City Abstract, LLC ALTA Seller's Settlement St...",13,13,0.998211,4.14,0.000000,ALTA SELLER’S STATEMENT


In [ ]:
for i in range(1, test_df['label'].count()-1):
  
  if (test_df['probability'][i] < threshold):
    if (int(test_df['label'][i]) not in single_list):
      if ((test_df['label'][i] != test_df['label'][i+1]) and (test_df['label'][i] != test_df['label'][i-1])):
        test_df['label'][i] = test_df['label'][i-1] 

<ipython-input-77-01278fcf3926>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['label'][i] = test_df['label'][i-1]


In [ ]:
test_df.head()

,page#,package_name,text,token,label,probability,cpu_usage (seconds),ram_usage (MB),Meaning
0,1,218248_NJ_CAMDEN,File No.: CAHD-612066\nAFFIDAVIT OF TITLE\nPUR...,3,3,0.999406,4.60,62.332928,AFF OF TITLE
1,2,218248_NJ_CAMDEN,| File No.: CAHD-612066\n| 7. Exceptions and A...,3,3,0.999380,4.17,606.781440,AFF OF TITLE
2,3,218248_NJ_CAMDEN,"City Abstract, LLC ALTA Buyer's Settlement Sta...",11,11,0.999020,4.23,62.074880,ALTA BUYER’S STATEMENT
3,4,218248_NJ_CAMDEN,Description\na Credit\nTitle - Lender's NJRB 5...,11,11,0.999001,4.18,26.009600,ALTA BUYER’S STATEMENT
4,5,218248_NJ_CAMDEN,"City Abstract, LLC ALTA Seller's Settlement St...",13,13,0.998211,4.14,0.000000,ALTA SELLER’S STATEMENT


In [ ]:
test_df['single_page_prediction'] = 0
test_df['single_page_prediction_probability'] = 0

In [ ]:
SINGLE_PAGE = []
PROB = []
for i in tqdm(test_df['label']):
  if i in single_list:
    input_text = clean_text(test_df[test_df['label']==i]['text'].tolist()[0])
    RES = get_prediction_SINGLE_PAGE(input_text)
    pred_text = RES['label']
    pred_prob = RES['probability']
    PROB.append(pred_prob)

    SINGLE_PAGE.append(pred_text)
  else:
    SINGLE_PAGE.append('')
    PROB.append('')


100%|██████████| 367/367 [00:05<00:00, 67.15it/s]


In [ ]:
test_df['single_page_prediction'] = SINGLE_PAGE
test_df['single_page_prediction_probability'] = PROB

In [ ]:
test_df.to_csv("/content/drive/MyDrive/Annotation Folder/Few_Packages/SINGLE_PAGE_PREDICTION.csv", index=False)

In [ ]:
final_df = test_df[~(test_df['single_page_prediction']=='')]
final_df.columns

Index(['page#', 'package_name', 'text', 'token', 'label', 'probability',
       'cpu_usage (seconds)', 'ram_usage (MB)', 'Meaning',
       'single_page_prediction', 'single_page_prediction_probability'],
      dtype='object')

In [ ]:
final_df.shape

(26, 11)

In [ ]:
final_df[['page#', 'package_name','token', 'label', 'probability', 'single_page_prediction','single_page_prediction_probability',
       'Meaning']].to_csv("Single_page_sample_prediction.csv", index=False)

In [ ]:
df_res[df_res['label'].isin(single_list)].shape

(26, 9)